In [1]:
# Download NHSN file from URL 

import urllib
dls = "https://www.cdc.gov/nhsn/xls/master-organism-com-commensals-lists.xlsx"
urllib.request.urlretrieve(dls, "NHSN_Organism_List.xlsx")  # For Python 3

import pandas as pd

# reading the NHSN file

df_NHSN = pd.read_excel("C:/Users/DixitV7/Downloads/Download-1/DataCollection/get_phones_data/NHSN_Organism_List.xlsx", sheet_name="All Organisms",header = 1)
df_NHSN_CC = pd.read_excel("C:/Users/DixitV7/Downloads/Download-1/DataCollection/get_phones_data/NHSN_Organism_List.xlsx", sheet_name="Common Commensals")
df_NHSN_MBI = pd.read_excel("C:/Users/DixitV7/Downloads/Download-1/DataCollection/get_phones_data/NHSN_Organism_List.xlsx", sheet_name="MBI Organisms")
df_NHSN_UTI = pd.read_excel("C:/Users/DixitV7/Downloads/Download-1/DataCollection/get_phones_data/NHSN_Organism_List.xlsx", sheet_name="UTI Bacteria")

# Appending all the data sheets into one
df= (pd.concat([df_NHSN,df_NHSN_CC,df_NHSN_MBI,df_NHSN_UTI],ignore_index=True, sort =False))

# Removing Duplicate rows from the file
df_NHSN_1 = df.drop_duplicates(subset =['NHSN Code','NHSN Display Name','SNOMED Preferred Term','SNOMED Code'],keep='last')


# Adding Common Commensal, MBI and UTI data in columns

result= []
for i in df_NHSN_1.index:
    jj= df_NHSN_1['NHSN Code'][i]
    for j in df_NHSN_CC.index:
        kk=df_NHSN_CC['NHSN Code'][j]
        if (jj == kk):
            result.append("Yes")
            break;
    if (jj != kk):
        result.append("No")
        
df_NHSN_1['Common Commensals']= result


result1= []
for i in df_NHSN_1.index:
    jj= df_NHSN_1['NHSN Code'][i]
    for j in df_NHSN_MBI.index:
        kk=df_NHSN_MBI['NHSN Code'][j]
        if (jj == kk):
            result1.append("Yes")
            break;
    if (jj != kk):
        result1.append("No")


df_NHSN_1['MBI']= result1


result2= []
for i in df_NHSN_1.index:
    jj= df_NHSN_1['NHSN Code'][i]
    for j in df_NHSN_UTI.index:
        kk=df_NHSN_UTI['NHSN Code'][j]
        if (jj == kk):
            result2.append("Yes")
            break;
    if (jj != kk):
        result2.append("No")
        
df_NHSN_1['UTI']= result2
        


C:\Users\DixitV7\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\DixitV7\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\DixitV7\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [2]:
## Running beautiful soup to extract the data from CDC tool kit file

import requests
from bs4 import BeautifulSoup

response = requests.get("https://www.cdc.gov/nhsn/cdaportal/toolkits.html").text

soup = BeautifulSoup(response, 'html.parser')


In [3]:
## Get the URL from the webpage so that the data can be downloaded in the machine
import urllib
urls = []
for link in soup.find_all('a'):
    urls.append(link.get('href'))
    

if '/nhsn/cdaportal/documents/toolkits/idm-vendors-9.5_10.0.xlsx' in urls:
    a= "https://www.cdc.gov/" + "/nhsn/cdaportal/documents/toolkits/idm-vendors-9.5_10.0.xlsx"
    dls = a
    urllib.request.urlretrieve(dls, "idm-vendors-9.5_10.0.xlsx") 

In [4]:
## Reading the data into Pandas DataFrame
df_NHSN_addcol1 = pd.read_excel("C:/Users/DixitV7/Downloads/Download-1/DataCollection/get_phones_data/idm-vendors-9.5_10.0.xlsx", sheet_name="Pathogen Codes 2021",header = 0)

In [5]:
## Renaming the columns to be in Sync
df_NHSN_addcol1=df_NHSN_addcol1.rename(columns = {'New Code':'NHSN Code','Term Type3':'Term_Type3'})

In [6]:
## Merging two datasets inner join applied

import pandas as pd
df_merge1= pd.merge(df_NHSN_1, df_NHSN_addcol1,on=["NHSN Code"])

In [7]:
## Mapping the Columns with relevant and meaningful information
def up_tt2(Term_Type3):
    if (Term_Type3 == 'P'):
        return "Preferred"
    elif (Term_Type3 == 'S'):
        return "Synonym"

df_merge1["Term_Type3"]=df_merge1["Term_Type3"].apply(up_tt2)

In [8]:
def up_abg(Antibiogram2):
    if (Antibiogram2 == 'O'):
        return "Optional"
    elif (Antibiogram2 == 'R'):
        return "Required"

df_merge1["Antibiogram2"]=df_merge1["Antibiogram2"].apply(up_abg)

In [9]:
## Deleting not relevant and Null values columns from the data

del df_merge1['MBI Pathogen']
del df_merge1['CC']
del df_merge1['Planned Version']
del df_merge1['NHSN Internal Sort']
del df_merge1['Defect/CR']
del df_merge1['Change Type']
del df_merge1['Description for Drop-down in App']
del df_merge1['SCTID (U.S. Edition 2018-09)\n\nNHSN 9.4 release\nJan 2020']
del df_merge1['SCTID (U.S. Edition 2019-09)\n\nNHSN 10.0 release\nJan 2021']
del df_merge1['SNOMED Preferred Term_y']
del df_merge1['SNOMED Fully Specified Name']
del df_merge1['Drug resistant organism']    

In [14]:
## Mapping of data 

def up_abgt(Type1):
     if (Type1 == 'B'):
        return "Bacteria"
     elif (Type1 == 'F'):
        return "Fungus"
     elif (Type1 == 'V'):
        return "Virus"
     elif (Type1 == 'P'):
        return "Parasite"

df_merge1["Type1"]=df_merge1["Type1"].apply(up_abgt)


In [10]:
## Changing data type of the column
df_merge1['SNOMED Code'] = df_merge1['SNOMED Code'].apply(lambda x: str(x))

In [16]:
## Renaming some of the column
df_NHSN_addcol1=df_NHSN_addcol1.rename(columns = {'LOS/MEN':'LOS_MEN','PS and LTCF Component Organisms':'PS_and_LTCF_Component_Organisms','VAE/PNEU included':'VAE_PNEU_included','LCBI Pathogen':'LCBI_Pathogen','ARO Pathogen':'ARO_Pathogen'})

In [11]:
## Mapping data with relevant information
df_merge1=df_merge1.replace('X','Yes')

In [12]:
## taking the copy in the local machine
df_merge1.to_csv('C:/Users/DixitV7/Downloads/Download-1/DataCollection/df_merge1.csv')

In [13]:
df_merge1.head()

,NHSN Code,NHSN Display Name,SNOMED Preferred Term_x,SNOMED Code,Common Commensals,MBI,UTI,Old Code,Code System,Term_Type3,...,LabSpCrGpC5,Type1,PathAntiB_Prior2021,PathAntiB_PS,PathAntiB_LTC,PathAntiB_DIAL,Unnamed: 29,Coag-neg staph,Viridans group strep,Enterobacteriaceae
0,ACANT,Acanthamoeba,Acanthamoeba,50875003,No,No,No,NaN,fullPathogen,Preferred,...,NaN,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PSEBOY*5,Acladium castellani,Pseudallescheria boydii,91088000,No,No,No,NaN,fullPathogen,Synonym,...,Yes,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SCEDBOYD*3,Acladium castellanii,Scedosporium boydii,24305000,No,No,No,NaN,fullPathogen,Synonym,...,Yes,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACRE,Acremonium,Acremonium,63996006,No,No,No,NaN,fullPathogen,Preferred,...,Yes,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACREALAB,Acremonium alabamensis,Acremonium alabamensis,83075006,No,No,No,NaN,fullPathogen,Preferred,...,Yes,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## Merge 2 process started 

In [14]:
## Transferring SNOMED Preferred Term into a list from the NHSN data set 

ls=df_NHSN_1['SNOMED Preferred Term'].to_list()

In [15]:
## Taking columns and adding them into the list
l=df_NHSN_1['SNOMED Code'].astype(str)+'--'+df_NHSN_1['SNOMED Preferred Term'].astype(str)

In [16]:
## Converting dataframe into list 
list1=l.values.tolist()
list1[0:3]

['50875003--Acanthamoeba',
 '91088000--Pseudallescheria boydii',
 '24305000--Scedosporium boydii']

In [43]:
## With the help of beautiful soup extracted the data for the SNOMED Code and Preferred term for the given microbioorganisms
## converted html data into dataframe and append and format the same in required format

import requests
import pandas as pd

from bs4 import BeautifulSoup

for i in range(0,1):
    j=list1[i]
    response = requests.get("https://www.findacode.com/snomed/" + j + ".html").text
    soup = BeautifulSoup(response, 'html')
    ab=soup.find_all('table',{'class':"edit-table"})
    lsteee = pd.read_html(str(ab))[0]
    lsteee_tr=lsteee.transpose()
    
from bs4 import BeautifulSoup
    
for k in range(1,len(list1)):
    m=list1[k]
    response = requests.get("https://www.findacode.com/snomed/" + m + ".html").text
    soup = BeautifulSoup(response, 'html')
    ab=soup.find_all('table',{'class':"edit-table"})
    lsteeef = pd.read_html(str(ab))[0]
    lsteee_tr1=lsteeef.transpose()
    lsteee_tr= (pd.concat([lsteee_tr,lsteee_tr1],ignore_index=True, sort =False))

lsteee_tr = lsteee_tr.drop_duplicates()


50875003--Acanthamoeba
91088000--Pseudallescheria boydii
24305000--Scedosporium boydii
63996006--Acremonium
83075006--Acremonium alabamensis
243484001--Acremonium kiliense
36128004--Acremonium potroni
36128004--Acremonium potroni
654006--Acremonium recifei
17720002--Acremonium strictum
45462000--Rhinocladiella aquaspersa
55038006--Chrysosporium
26861000146102--Scedosporium apiospermum
91088000--Pseudallescheria boydii
75059003--Alternaria
36703000--Alternaria alternata
28387003--Alternaria dianthicola
21702003--Alternaria infectoria
36703000--Alternaria alternata
35667002--Alternaria tenuissima
17113000--Ancylostoma
7153002--Angiostrongylus
68038008--Apophysomyces
71235006--Apophysomyces elegans
28375005--Flavivirus
58234003--Arenavirus
11860003--Arthroderma
112406004--Arthrographis
66953006--Ascaris
2429008--Aspergillus
21477007--Aspergillus candidus
87240002--Aspergillus clavatus
4204003--Aspergillus deflectus
54120001--Aspergillus flavipes
66821001--Aspergillus flavus
32684000--Aspe

42797005--Leptosphaeria
48529007--Leptosphaeria senegalensis
80279006--Leptosphaeria tompkinsii
90959007--Loboa
448291001--Lodderomyces
449485005--Lodderomyces elongisporus
82518003--Lymphocytic choriomeningitis virus
75059003--Alternaria
23009007--Madurella
82871006--Madurella grisea
24876009--Madurella mycetomatis
24876009--Madurella mycetomatis
18097004--Malassezia
29619007--Malassezia furfur
707291006--Malassezia globosa
29619007--Malassezia furfur
51257000--Malassezia pachydermatis
10171000087105--Malassezia restricta
447178007--Malassezia sympodialis
87186007--Malbranchea
424421007--Marburgvirus
38350001--Mastadenovirus
52584002--Measles virus
116443004--Metarhizium
116446007--Metarhizium anisopliae
718505003--Meyerozyma
29812000--Meyerozyma guilliermondii
243662007--Microsporidium
11983001--Microsporum
243547001--Sarcinosporon inkin
84489001--Mold
68855006--Molluscum contagiosum virus
59774002--Monkeypox virus
26861000146102--Scedosporium apiospermum
83313003--Genus Morbilliviru

88274000--Trypanosoma cruzi
72637009--Trypanosoma rangeli
88274000--Trypanosoma cruzi
65789000--Ulocladium
49086004--Ustilago
13126002--Vaccinia virus
80298008--Varicellovirus
19551004--Human herpesvirus 3
80298008--Varicellovirus
243568000--Variola virus
62958000--Venezuelan equine encephalomyelitis virus
50734003--Verticillium
49872002--Virus
70200002--Wangiella
20586007--Wangiella dermatitidis
57311007--West Nile virus
11428003--Western equine encephalomyelitis virus
445325002--Xenotropic murine leukemia virus-related virus
62093005--Yeast
26630006--Yellow fever virus
50471002--Zika virus
63775007--Candida lipolytica
3265006--Candida
53326005--Candida albicans
20849004--Candida stellatoidea
3491000146109--Candida auris
3521000146107--Candida bracarensis
283868009--Candida catenulata
283868009--Candida catenulata
283867004--Candida ciferrii
115952006--Candida dubliniensis
4001000146106--Candida duobushaemulonii
444877006--Candida glabrata
16557001--Candida guilliermondii
283874009--C

704975008--Trueperella bernardiae
44723000--Arcanobacterium haemolyticum
428939003--Arcanobacterium pluranimalium
258933002--Trueperella pyogenes
114206000--Arcobacter
114207009--Arcobacter butzleri
408629003--Arcobacter nitrofigilis
428986004--Arsenicicoccus
428067009--Arsenicicoccus bolidensis
56214009--Arthrobacter
113432004--Arthrobacter agilis
127496003--Arthrobacter albus
428949000--Arthrobacter arilaitensis
113433009--Arthrobacter astrocyaneus
113434003--Arthrobacter aurescens
428950000--Arthrobacter bergerei
428038008--Arthrobacter chlorophenolicus
44955005--Arthrobacter citreus
417608004--Arthrobacter creatinolyticus
113435002--Arthrobacter crystallopoietes
113436001--Arthrobacter cumminsii
429762004--Arthrobacter flavus
428332000--Arthrobacter gandavensis
429237000--Arthrobacter gangotriensis
3840003--Arthrobacter globiformis
113437005--Arthrobacter histidinolovorans
48276002--Arthrobacter ilicis
429238005--Arthrobacter kerguelensis
427847001--Arthrobacter koreensis
127497007

114168007--Dichelobacter nodosus
10091000087105--Bacteroides nordii
4256006--Capnocytophaga ochracea
114146002--Prevotella oralis
114147006--Prevotella oris
419466004--Prevotella oulorum
86936008--Bacteroides ovatus
114134001--Prevotella buccae
75376009--Tissierella praeacuta
426435007--Alistipes putredinis
50504006--Bacteroides pyogenes
115949003--Prevotella ruminicola
114166006--Porphyromonas macacae
446487001--Odoribacter splanchnicus
163009--Bacteroides stercoris
50504006--Bacteroides pyogenes
50504006--Bacteroides pyogenes
34236001--Bacteroides thetaiotaomicron
59604009--Bacteroides uniformis
450602004--Campylobacter ureolyticus
114152001--Prevotella veroralis
52798008--Bacteroides vulgatus
425729002--Bacteroides zoogleoformans
416418007--Balneatrix
416691007--Balneatrix alpica
62496009--Bartonella
243352007--Bartonella bacilliformis
114223009--Bartonella clarridgeiae
103513003--Bartonella elizabethae
103510000--Bartonella henselae
103511001--Bartonella quintana
103512008--Bartone

...

53932007--Gram-negative coccobacillus
18383003--Gram-negative coccus
83410001--Gram-negative diplococcus
73460001--Gram-negative pleomorphic bacillus
83514008--Gram-positive bacillus
49682003--Gram positive coccobacillus
59206002--Gram-positive coccus
11471007--Gram-positive diplococcus
243288000--Gram-variable bacillus
301782009--Gram-variable coccobacillus
301781002--Gram-variable coccus
409814005--Granulicatella
113713009--Granulicatella adiacens
115944008--Granulicatella elegans
431444000--Grimontia
433669003--Grimontia hollisae
71268004--Haemophilus
442680008--Aggregatibacter actinomycetemcomitans
27784005--Haemophilus aegyptius
442683005--Aggregatibacter aphrophilus
72339004--Haemophilus haemoglobinophilus
64889004--Haemophilus ducreyi
72339004--Haemophilus haemoglobinophilus
54950008--Haemophilus haemolyticus
44470000--Haemophilus influenzae
89800007--Haemophilus parahaemolyticus
51593004--Haemophilus parainfluenzae
432919004--Haemophilus pittmaniae
442651004--Aggregatibacter se

114203008--Microbacterium imperiale
114204002--Microbacterium lacticum
114063000--Microbacterium liquefaciens
414710003--Microbacterium maritypicum
414713001--Microbacterium oxydans
414714007--Microbacterium paraoxydans
414716009--Microbacterium resistens
52864008--Micrococcus
416555005--Micrococcus antarcticus
48299003--Micrococcus luteus
113770004--Nesterenkonia halobia
113772007--Kocuria kristinae
48299003--Micrococcus luteus
61376003--Micrococcus lylae
48299003--Micrococcus luteus
112344007--Peptococcus niger
67084000--Micrococcus nishinomiyaensis
113773002--Kocuria rosea
113775009--Kytococcus sedentarius
416958001--Micromonospora
430840008--Micromonospora chalcea
70366009--Nocardia brevicatena
24231001--Mitsuokella
114137008--Prevotella dentalis
55553008--Mitsuokella multacida
8984004--Mobiluncus
30224004--Mobiluncus curtisii
13716002--Mobiluncus mulieris
71655003--Moellerella
37107002--Moellerella wisconsensis
416887004--Mogibacterium
429898003--Mogibacterium diversum
429899006--

426054003--Nocardia asiatica
30092002--Nocardia asteroides
414864009--Nocardia asteroides complex
430848001--Nocardia beijingensis
58970008--Nocardia brasiliensis
70366009--Nocardia brevicatena
61208001--Nocardia carnea
63087008--Nocardia otitidis caviarum
414866006--Nocardia cyriacigeorgica
418717009--Nocardia elegans
85516007--Nocardia farcinica
430654003--Nocardia higoensis
430770006--Nocardia inohanensis
450468006--Nocardia kruczakiae
855085291000087109--Nocardia mexicana
430772003--Nocardia niigatensis
37621001--Nocardia nova
427705005--Amycolatopsis orientalis
63087008--Nocardia otitidis caviarum
63087008--Nocardia otitidis caviarum
414867002--Nocardia paucivorans
431063007--Nocardia pneumoniae
414868007--Nocardia pseudobrasiliensis
430672009--Nocardia sienata
430588000--Nocardia testacea
80379008--Nocardia transvalensis
414869004--Nocardia veterana
430661004--Nocardia yamanashiensis
23593005--Nocardiopsis
15526005--Nocardiopsis dassonvillei
416314008--Nocardiopsis synnemataforma

58024003--Pseudomonas stutzeri
17191002--Pseudomonas pertucinogena
114053008--Ralstonia pickettii
30955004--Pseudomonas pseudoalcaligenes
116399000--Burkholderia pseudomallei
68608003--Pseudomonas putida
113700003--Shewanella putrefaciens
113677007--Burkholderia pyrrocinia
58024003--Pseudomonas stutzeri
43721005--Comamonas testosteroni
113940001--Pseudomonas veronii
113690000--Brevundimonas vesicularis
115131002--Pseudoramibacter
113828002--Pseudoramibacter alactolyticus
428474009--Pseudoxanthomonas
698063009--Pseudoxanthomonas koreensis
709302003--Psychrobacillus
34412002--Psychrobacter
70465007--Psychrobacter immobilis
113941002--Psychrobacter phenylpyruvicus
48569008--Rahnella
10021006--Rahnella aquatilis
415234001--Ralstonia
416329005--Cupriavidus gilardii
415235000--Ralstonia insidiosa
415236004--Ralstonia mannitolilytica
114053008--Ralstonia pickettii
416495009--Cupriavidus respiraculi
417468005--Raoultella
416832000--Raoultella ornithinolytica
416264004--Raoultella planticola
41

76199005--Streptococcus constellatus
415601009--Streptococcus constellatus subspecies constellatus
415602002--Streptococcus constellatus subspecies pharyngis
34838009--Streptococcus criceti
113980005--Streptococcus cristatus
113714003--Abiotrophia defectiva
708952006--Streptococcus dentisani
12704003--Streptococcus dolonei
46464008--Enterococcus durans
40812000--Streptococcus dysgalactiae
113982002--Streptococcus dysgalactiae subsp. dysgalactiae
128338004--Streptococcus dysgalactiae subsp. equisimilis
115944008--Granulicatella elegans
437956001--Streptococcus entericus
12447002--Streptococcus equi
113984001--Streptococcus equi subsp. equi
33708000--Streptococcus equi subsp. zooepidemicus
72646003--Streptococcus equinus
78065002--Enterococcus faecalis
90272000--Enterococcus faecium
10049004--Streptococcus ferus
53233007--Enterococcus gallinarum
113985000--Streptococcus gallolyticus
421544007--Streptococcus gallolyticus subspecies gallolyticus
421004005--Streptococcus gallolyticus subspe

In [ ]:
## Mapped data to local machine
lsteee_tr.to_csv('C:/Users/DixitV7/Downloads/Download-1/DataCollection/SNOMET_CT_Map.csv')

In [17]:
## Reading the downloaded data

df_nn1= pd.read_csv("C:/Users/DixitV7/Downloads/Download-1/DataCollection/SNOMED_CT_MAP.csv", header = 1)


In [18]:
## Below helps to format and map the data that will be useful in joining with Merge1

df_nn1_nt_Nan= df_nn1.dropna()
df_nn1_is_Nan=df_nn1[df_nn1.isnull().any(axis=1)]

In [19]:
del df_nn1_is_Nan['cpt crosswalks']
df_nn1_is_Nan=df_nn1_is_Nan.rename(columns= {'children':'hierarchies','hierarchies':'ancestors','ancestors':'cpt crosswalks'})
df_nn1_is_Nan.insert(loc = 8,column = 'children',value = "")


In [20]:
df_merge2nd = (pd.concat([df_nn1_nt_Nan,df_nn1_is_Nan],ignore_index=True, sort =False))

In [21]:
del df_merge2nd['cpt crosswalks']
df_merge2nd[['FirstH','ancestorsH']] = df_merge2nd.ancestors.str.split("sorted most to least specific",expand=True)
df_merge2nd[['SecondH','hierarchiesH']] = df_merge2nd.hierarchies.str.split("a selection of possible paths",expand=True)
del df_merge2nd['FirstH']
del df_merge2nd['SecondH']
del df_merge2nd['ancestors']
del df_merge2nd['hierarchies']
del df_merge2nd['0']

In [22]:
## Removing duplicates from the data
df_merge2nd = df_merge2nd.drop_duplicates(subset =['SNOMED code','name','status','date introduced','fully specified name(s)','synonyms','parents','children','ancestorsH','hierarchiesH'],keep='last')

In [23]:
df_merge2nd=df_merge2nd.rename(columns = {'SNOMED code':'SNOMED Code'})

df_merge2nd.to_csv('C:/Users/DixitV7/Downloads/Download-1/DataCollection/df_merge2nd.csv')

In [24]:
## Merging the data to new data frame Merge3 

df_merge3= pd.merge(df_merge1,df_merge2nd.drop_duplicates(),how='inner',on='SNOMED Code')

In [25]:
df_merge3.head()

,NHSN Code,NHSN Display Name,SNOMED Preferred Term_x,SNOMED Code,Common Commensals,MBI,UTI,Old Code,Code System,Term_Type3,...,Enterobacteriaceae,name,status,date introduced,fully specified name(s),synonyms,parents,children,ancestorsH,hierarchiesH
0,ACANT,Acanthamoeba,Acanthamoeba,50875003,No,No,No,NaN,fullPathogen,Preferred,...,NaN,Acanthamoeba,active,2002-01-31,Acanthamoeba (organism),Acanthamoeba speciesGenus: AcanthamoebaAcantha...,Acanthamoebidae 106599006,Acanthameba astronyxise 302556001 removed: 200...,Acanthamoebidae 106599006Order Amoebida 106597...,SNOMED CT Concept 138875005 Organism 410607006...
1,PSEBOY*5,Acladium castellani,Pseudallescheria boydii,91088000,No,No,No,NaN,fullPathogen,Synonym,...,NaN,Pseudallescheria boydii,active,2002-01-31,Pseudallescheria boydii (organism),Pseudoallescheria boydiiAllescheria boydiiPetr...,Pseudoallescheria 54930009Pseudallescheria boy...,,Pseudoallescheria 54930009Pseudallescheria boy...,SNOMED CT Concept 138875005 Organism 410607006...
2,PSEBOY*2,Allescheria boydii,Pseudallescheria boydii,91088000,No,No,No,NaN,fullPathogen,Synonym,...,NaN,Pseudallescheria boydii,active,2002-01-31,Pseudallescheria boydii (organism),Pseudoallescheria boydiiAllescheria boydiiPetr...,Pseudoallescheria 54930009Pseudallescheria boy...,,Pseudoallescheria 54930009Pseudallescheria boy...,SNOMED CT Concept 138875005 Organism 410607006...
3,PSEBOY*1,Petriellidium boydii,Pseudallescheria boydii,91088000,No,No,No,PETRI,fullPathogen,Synonym,...,NaN,Pseudallescheria boydii,active,2002-01-31,Pseudallescheria boydii (organism),Pseudoallescheria boydiiAllescheria boydiiPetr...,Pseudoallescheria 54930009Pseudallescheria boy...,,Pseudoallescheria 54930009Pseudallescheria boy...,SNOMED CT Concept 138875005 Organism 410607006...
4,PSEBOY*4,Petriellidum boydii,Pseudallescheria boydii,91088000,No,No,No,NaN,fullPathogen,Synonym,...,NaN,Pseudallescheria boydii,active,2002-01-31,Pseudallescheria boydii (organism),Pseudoallescheria boydiiAllescheria boydiiPetr...,Pseudoallescheria 54930009Pseudallescheria boy...,,Pseudoallescheria 54930009Pseudallescheria boy...,SNOMED CT Concept 138875005 Organism 410607006...


In [26]:
## Mapping data to local machine

df_merge3.to_csv('C:/Users/DixitV7/Downloads/Download-1/DataCollection/df_merge3.csv')

In [27]:
## Adding a new column to dataframe by converting digits as a deimiter

df_merge3['hie']=df_merge3['hierarchiesH'].str.replace('[0-9]{1,9}', ':')

In [28]:
## Adding new cloumns for Domain and Kingdom for Microbioorganism

df_merge3[['hie1','hie2','Domain','Kingdom','hie5']]=df_merge3['hie'].str.split(':',n=4 ,expand = True)
del df_merge3['hie']
del df_merge3['hie1']
del df_merge3['hie2']


In [29]:
df_merge3['Domain']=df_merge3['Domain'].str.replace('Domain Eukarya', 'Eukarya')

In [30]:
df_merge3['Kingdom']=df_merge3['Kingdom'].str.replace('Kingdom', '')

In [31]:
df_merge3.to_csv('C:/Users/DixitV7/Downloads/Download-1/DataCollection/df_merge3_Test.csv')

In [32]:
df_merge3[['hie6','hie7']]=df_merge3['hie5'].str.split(':',n=1 ,expand = True)

In [33]:
df_merge3[['Division','hie7']]=df_merge3['hie6'].str.split('Class',n=1 ,expand = True)

    

In [34]:
## Removing non relevant columns from the Dataframe

del df_merge3['name']
del df_merge3['ancestorsH']
del df_merge3['hierarchiesH']
del df_merge3['hie5']
del df_merge3['hie6']
del df_merge3['hie7']


In [35]:
df_merge3.head()

,NHSN Code,NHSN Display Name,SNOMED Preferred Term_x,SNOMED Code,Common Commensals,MBI,UTI,Old Code,Code System,Term_Type3,...,Enterobacteriaceae,status,date introduced,fully specified name(s),synonyms,parents,children,Domain,Kingdom,Division
0,ACANT,Acanthamoeba,Acanthamoeba,50875003,No,No,No,NaN,fullPathogen,Preferred,...,NaN,active,2002-01-31,Acanthamoeba (organism),Acanthamoeba speciesGenus: AcanthamoebaAcantha...,Acanthamoebidae 106599006,Acanthameba astronyxise 302556001 removed: 200...,Eukarya,Protozoa,Sarcodina
1,PSEBOY*5,Acladium castellani,Pseudallescheria boydii,91088000,No,No,No,NaN,fullPathogen,Synonym,...,NaN,active,2002-01-31,Pseudallescheria boydii (organism),Pseudoallescheria boydiiAllescheria boydiiPetr...,Pseudoallescheria 54930009Pseudallescheria boy...,,Eukarya,Fungi,Ascomycota
2,PSEBOY*2,Allescheria boydii,Pseudallescheria boydii,91088000,No,No,No,NaN,fullPathogen,Synonym,...,NaN,active,2002-01-31,Pseudallescheria boydii (organism),Pseudoallescheria boydiiAllescheria boydiiPetr...,Pseudoallescheria 54930009Pseudallescheria boy...,,Eukarya,Fungi,Ascomycota
3,PSEBOY*1,Petriellidium boydii,Pseudallescheria boydii,91088000,No,No,No,PETRI,fullPathogen,Synonym,...,NaN,active,2002-01-31,Pseudallescheria boydii (organism),Pseudoallescheria boydiiAllescheria boydiiPetr...,Pseudoallescheria 54930009Pseudallescheria boy...,,Eukarya,Fungi,Ascomycota
4,PSEBOY*4,Petriellidum boydii,Pseudallescheria boydii,91088000,No,No,No,NaN,fullPathogen,Synonym,...,NaN,active,2002-01-31,Pseudallescheria boydii (organism),Pseudoallescheria boydiiAllescheria boydiiPetr...,Pseudoallescheria 54930009Pseudallescheria boy...,,Eukarya,Fungi,Ascomycota


In [36]:
NHSN_Organism_DCP=df_merge3
NHSN_Organism_DCP.to_csv('C:/Users/DixitV7/Downloads/Download-1/DataCollection/NHSN_Organism_DCP.csv')